## Первичная обработка данных

В этом ноутбуке проводится первичный анализ и обработка данных, включая:
- Загрузка и обзор данных
- Очистка данных от пропусков и аномалий
- Преобразование данных для дальнейшего анализа

Далее подготовленные данные будут использоваться для построения моделей машинного обучения и проведения статистического анализа.

Данные представлены в формате .tsv (табуляция-разделитель) и представляют собой описание различных характеристик объектов исследования.

Объектом исследования являются данные fMRI (функциональная магнитно-резонансная томография) пациентов с СДВГ и контрольной группы. Так же в датасете присутствуют столбцы с указанием возраста испытуемых, их пола и наличия диагноза СДВГ с разделением на подтипы: в основном гиперактивный, в основном невнимательный и комбинированный. И группа контроля.

Данные предполагается отфильтровать по возрасту испытуемых и наличию диагноза СДВГ. Все строки, где отсутствует информация о возрасте или диагнозе, будут удалены из датасета.
Когортами исследования являются подростки в возрасте от 12 до 16 лет (не включая 16-тилетний возраст) и молодые взрослые от 16 до 21 года.

Результатом первичной обработки будут являться очищенные и отфильтрованные датасеты, содержащие только релевантные данные для дальнейшего анализа. Датасеты будут разделены на две группы по возрасту: подростки и молодые взрослые.

Данные были собраны в рамках исследования, проведенного в Национальном институте здоровья (NIH) и доступны для общественности через платформу OpenNeuro.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Настраиваем отображение графиков в Jupyter Notebook
%matplotlib inline
sns.set(style="whitegrid")

# Настраиваем отображение всех колонок DataFrame
pd.set_option('display.max_columns', 100)  # Ограничение по колонкам для удобства
pd.set_option('display.max_rows', 1000)  # Ограничение по строкам для удобства
pd.set_option('display.width', 1000)  # Ширина отображения

In [2]:
TEEN_MINIMAL_AGE = 12
TEEN_MAXIMAL_AGE = 15.99
ADULT_MINIMAL_AGE = 16
ADULT_MAXIMAL_AGE = 21.99

### 1. Загрузка данных

In [3]:
# Укажите правильный путь к вашему файлу
file_path = 'adhd200_preprocessed_phenotypics.tsv'

# Загружаем .tsv файл в DataFrame с обработкой ошибки
try:
    df = pd.read_csv(file_path, sep='\t')
    print(f"Файл '{file_path}' успешно загружен.")
except FileNotFoundError:
    print(f"Ошибка: Файл '{file_path}' не найден. Пожалуйста, убедитесь, что он находится в правильной директории.")

Файл 'adhd200_preprocessed_phenotypics.tsv' успешно загружен.


### 2. Первичный осмотр данных

In [4]:
print("\n--- 2. Первичный осмотр данных ---")
    
# Вывод первых 5 строк для общего представления
print("\nПервые 5 строк данных:")
print(df.head())


--- 2. Первичный осмотр данных ---

Первые 5 строк данных:
   ScanDir ID  Site  Gender    Age Handedness DX   Secondary Dx ADHD Measure ADHD Index Inattentive Hyper/Impulsive  IQ Measure  Verbal IQ  Performance IQ  Full2 IQ  Full4 IQ Med Status  QC_Athena  QC_NIAK
0     2371032     3     0.0  10.73          1  0            NaN            2         47          55              43         1.0      121.0           119.0       NaN     122.0          1        1.0      1.0
1     2026113     3     0.0  12.99          1  1            NaN            2         90          89              78         1.0      122.0           108.0       NaN     106.0          1        1.0      1.0
2     3434578     3     0.0   8.12          1  0            NaN            2         42          42              43         1.0       85.0            98.0       NaN      89.0          1        1.0      1.0
3     8628223     3     0.0  10.81          1  0  Simple phobia            2         42          49              49 

In [5]:
# Вывод информации о типах данных и наличии пропусков
print("\nИнформация о DataFrame (типы данных, пропуски):")
df.info()


Информация о DataFrame (типы данных, пропуски):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ScanDir ID       973 non-null    int64  
 1   Site             973 non-null    int64  
 2   Gender           972 non-null    float64
 3   Age              973 non-null    float64
 4   Handedness       970 non-null    object 
 5   DX               973 non-null    object 
 6   Secondary Dx     157 non-null    object 
 7   ADHD Measure     655 non-null    object 
 8   ADHD Index       613 non-null    object 
 9   Inattentive      692 non-null    object 
 10  Hyper/Impulsive  692 non-null    object 
 11  IQ Measure       899 non-null    float64
 12  Verbal IQ        725 non-null    float64
 13  Performance IQ   725 non-null    float64
 14  Full2 IQ         164 non-null    float64
 15  Full4 IQ         898 non-null    float64
 16  Med Status   

In [6]:
# Подсчет количества пропущенных значений в каждом столбце
print("\nКоличество пропущенных значений в каждом столбце:")
print(df.isnull().sum())


Количество пропущенных значений в каждом столбце:
ScanDir ID           0
Site                 0
Gender               1
Age                  0
Handedness           3
DX                   0
Secondary Dx       816
ADHD Measure       318
ADHD Index         360
Inattentive        281
Hyper/Impulsive    281
IQ Measure          74
Verbal IQ          248
Performance IQ     248
Full2 IQ           809
Full4 IQ            75
Med Status         309
QC_Athena            8
QC_NIAK              8
dtype: int64


### 3. Предварительная обработка данных

Требуется выполнить следующие шаги:
- Проверить наличие и обработать пропущенные значения.
- Преобразовать категориальные переменные в числовые (если необходимо).
- Нормализовать или стандартизировать числовые переменные (если необходимо).

В частности, требуется удалить все строки с пропущенными значениями в столбцах 'DX' и 'Age' так как эти столбцы содержат критическую информацию.

In [7]:
# Удалим все строки с пропущенными значениями в столбцах 'DX' и 'Age'
df_final = df.dropna(subset=['DX', 'Age'], inplace=False)

# Удалим все строки со значеним 'pending' в столбцах 'DX' и 'Age'
df_final = df_final[df_final['DX'] != 'pending']
df_final = df_final[df_final['Age'] != 'pending']

# Преобразуем столбец 'Age' в числовой тип
df_final['Age'] = pd.to_numeric(df_final['Age'], errors='coerce')

# Выводим размер данных после удаления строк с пропущенными значениями
print(f"\nРазмер данных после удаления строк с пропущенными значениями в 'DX' и 'Age': {df_final.shape}")
print("\nПервые 5 строк данных после удаления:")
print(df_final.head())



Размер данных после удаления строк с пропущенными значениями в 'DX' и 'Age': (947, 19)

Первые 5 строк данных после удаления:
   ScanDir ID  Site  Gender    Age Handedness DX   Secondary Dx ADHD Measure ADHD Index Inattentive Hyper/Impulsive  IQ Measure  Verbal IQ  Performance IQ  Full2 IQ  Full4 IQ Med Status  QC_Athena  QC_NIAK
0     2371032     3     0.0  10.73          1  0            NaN            2         47          55              43         1.0      121.0           119.0       NaN     122.0          1        1.0      1.0
1     2026113     3     0.0  12.99          1  1            NaN            2         90          89              78         1.0      122.0           108.0       NaN     106.0          1        1.0      1.0
2     3434578     3     0.0   8.12          1  0            NaN            2         42          42              43         1.0       85.0            98.0       NaN      89.0          1        1.0      1.0
3     8628223     3     0.0  10.81          1  0 

### 3.1 Преобразование столбца Gender
Значения пола (Gender) преобразуем из числовых кодов в строковые обозначения:
- 1.0 (или 1) → 'M'
- 0.0 (или 0) → 'F'

Также предусмотрена обработка строковых представлений ('1', '0'). Если встречаются другие значения, они будут отмечены как NaN для последующего анализа (при необходимости можно будет обработать отдельно).

In [8]:
# Преобразуем столбец 'Gender' в значения 'M' и 'F'
# Допускаем варианты типов: float, int, str
if 'Gender' in df_final.columns:
    gender_map = {1: 'M', 1.0: 'M', '1': 'M', 0: 'F', 0.0: 'F', '0': 'F'}
    # Создадим копию столбца для контроля
    before_unique = df_final['Gender'].unique()
    df_final['Gender'] = df_final['Gender'].map(gender_map)
    print("Уникальные значения Gender до преобразования:", before_unique)
    after_unique = df_final['Gender'].unique()
    print("Уникальные значения Gender после преобразования:", after_unique)
    # Подсчет пропусков после маппинга (если были неожиданные значения)
    unmapped_count = df_final['Gender'].isna().sum()
    if unmapped_count > 0:
        print(f"Предупреждение: {unmapped_count} знач. не удалось сопоставить и они стали NaN.")
else:
    print("Столбец 'Gender' не найден в df_final.")

Уникальные значения Gender до преобразования: [ 0.  1. nan]
Уникальные значения Gender после преобразования: ['F' 'M' nan]
Предупреждение: 1 знач. не удалось сопоставить и они стали NaN.


In [9]:
# Вывод информации о типах данных и наличии пропусков
print("\nИнформация о DataFrame (типы данных, пропуски):")
df_final.info()


Информация о DataFrame (типы данных, пропуски):
<class 'pandas.core.frame.DataFrame'>
Index: 947 entries, 0 to 972
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ScanDir ID       947 non-null    int64  
 1   Site             947 non-null    int64  
 2   Gender           946 non-null    object 
 3   Age              947 non-null    float64
 4   Handedness       944 non-null    object 
 5   DX               947 non-null    object 
 6   Secondary Dx     131 non-null    object 
 7   ADHD Measure     629 non-null    object 
 8   ADHD Index       587 non-null    object 
 9   Inattentive      666 non-null    object 
 10  Hyper/Impulsive  666 non-null    object 
 11  IQ Measure       899 non-null    float64
 12  Verbal IQ        699 non-null    float64
 13  Performance IQ   699 non-null    float64
 14  Full2 IQ         164 non-null    float64
 15  Full4 IQ         872 non-null    float64
 16  Med Status       6

In [10]:
# Подсчет количества пропущенных значений в каждом столбце после очистки
print("\nКоличество пропущенных значений в каждом столбце после очистки:")
print(df_final.isnull().sum())


Количество пропущенных значений в каждом столбце после очистки:
ScanDir ID           0
Site                 0
Gender               1
Age                  0
Handedness           3
DX                   0
Secondary Dx       816
ADHD Measure       318
ADHD Index         360
Inattentive        281
Hyper/Impulsive    281
IQ Measure          48
Verbal IQ          248
Performance IQ     248
Full2 IQ           783
Full4 IQ            75
Med Status         309
QC_Athena            8
QC_NIAK              8
dtype: int64


### 4. Определение возрастных групп

In [11]:
# Удалим из df_final все строки, где в столбце 'Age' значение меньше или равно TEEN_MINIMAL_AGE
df_final = df_final[df_final['Age'] > TEEN_MINIMAL_AGE]
# Возрастные группы: подростки (TEEN_MINIMAL_AGE - TEEN_MAXIMAL_AGE), молодые взрослые (ADULT_MINIMAL_AGE - ADULT_MAXIMAL_AGE)
df_final['AgeGroup'] = pd.cut(df_final['Age'], bins=[TEEN_MINIMAL_AGE, TEEN_MAXIMAL_AGE, ADULT_MAXIMAL_AGE], labels=['Teen', 'YoungAdult'])

print(f"\nРазмер данных после удаления строк с возрастом < {TEEN_MINIMAL_AGE}: {df_final.shape}")
print("\nПервые 5 строк данных после удаления:")
print(df_final.head())



Размер данных после удаления строк с возрастом < 12: (394, 20)

Первые 5 строк данных после удаления:
    ScanDir ID  Site Gender    Age Handedness DX   Secondary Dx ADHD Measure ADHD Index Inattentive Hyper/Impulsive  IQ Measure  Verbal IQ  Performance IQ  Full2 IQ  Full4 IQ Med Status  QC_Athena  QC_NIAK AgeGroup
1      2026113     3      F  12.99          1  1            NaN            2         90          89              78         1.0      122.0           108.0       NaN     106.0          1        1.0      1.0     Teen
4      1623716     3      F  12.65          1  1            NaN            2         87          90              90         1.0       89.0            88.0       NaN      89.0          1        1.0      1.0     Teen
5      1594156     3      M  12.87          1  0  Simple Phobia            2         54          55              57         1.0      124.0           104.0       NaN     108.0          1        1.0      1.0     Teen
10     2917777     3      M  12.66   

In [12]:
# Вывод информации о типах данных и наличии пропусков
print("\nИнформация о DataFrame (типы данных, пропуски):")
df_final.info()


Информация о DataFrame (типы данных, пропуски):
<class 'pandas.core.frame.DataFrame'>
Index: 394 entries, 1 to 972
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   ScanDir ID       394 non-null    int64   
 1   Site             394 non-null    int64   
 2   Gender           394 non-null    object  
 3   Age              394 non-null    float64 
 4   Handedness       392 non-null    object  
 5   DX               394 non-null    object  
 6   Secondary Dx     54 non-null     object  
 7   ADHD Measure     208 non-null    object  
 8   ADHD Index       217 non-null    object  
 9   Inattentive      217 non-null    object  
 10  Hyper/Impulsive  217 non-null    object  
 11  IQ Measure       349 non-null    float64 
 12  Verbal IQ        300 non-null    float64 
 13  Performance IQ   300 non-null    float64 
 14  Full2 IQ         110 non-null    float64 
 15  Full4 IQ         324 non-null    float64 
 16  

### 5. Выделение подгрупп по диагнозу и возрасту

In [13]:
# Разделим датасет на подгруппы по возрасту
teen_df = df_final[df_final['AgeGroup'] == 'Teen']
young_adult_df = df_final[df_final['AgeGroup'] == 'YoungAdult']

# Удалим столбец 'AgeGroup', так как он больше не нужен
teen_df = teen_df.drop(columns=['AgeGroup'])
young_adult_df = young_adult_df.drop(columns=['AgeGroup'])

print(f"\nРазмер данных для подростков ({TEEN_MINIMAL_AGE}-{TEEN_MAXIMAL_AGE}): {teen_df.shape}")
print(f"Размер данных для молодых взрослых ({ADULT_MINIMAL_AGE}-{ADULT_MAXIMAL_AGE}): {young_adult_df.shape}")

# Вывод первых 5 строк каждой подгруппы для проверки
print("\nПервые 5 строк данных для подростков:")
print(teen_df.head())

print("\nПервые 5 строк данных для молодых взрослых:")
print(young_adult_df.head())

# Убедимся, что в каждой подгруппе есть данные по диагнозам
print("\nРаспределение диагнозов в группе подростков:")
print(teen_df['DX'].value_counts())

print("\nРаспределение диагнозов в группе молодых взрослых:")
print(young_adult_df['DX'].value_counts())


Размер данных для подростков (12-15.99): (274, 19)
Размер данных для молодых взрослых (16-21.99): (117, 19)

Первые 5 строк данных для подростков:
    ScanDir ID  Site Gender    Age Handedness DX   Secondary Dx ADHD Measure ADHD Index Inattentive Hyper/Impulsive  IQ Measure  Verbal IQ  Performance IQ  Full2 IQ  Full4 IQ Med Status  QC_Athena  QC_NIAK
1      2026113     3      F  12.99          1  1            NaN            2         90          89              78         1.0      122.0           108.0       NaN     106.0          1        1.0      1.0
4      1623716     3      F  12.65          1  1            NaN            2         87          90              90         1.0       89.0            88.0       NaN      89.0          1        1.0      1.0
5      1594156     3      M  12.87          1  0  Simple Phobia            2         54          55              57         1.0      124.0           104.0       NaN     108.0          1        1.0      1.0
10     2917777     3      M 

In [ ]:
# Сохраним подготовленные данные в новые файлы для дальнейшего анализа (перезапишем существующие файлы)
teen_df.to_csv('cohort_teen_participants.csv', index=False)
young_adult_df.to_csv('cohort_adult_participants.csv', index=False)

Далее необходимо выделить подгруппы в каждой возрастной группе:
- Подростки с СДВГ
- Подростки без СДВГ (контрольная группа)
- Молодые взрослые с СДВГ
- Молодые взрослые без СДВГ (контрольная группа)
Для этого нужно отфильтровать данные по столбцу 'DX', где значения '1, 2, 3' указывают на наличие диагноза СДВГ, а '0' - на отсутствие диагноза.

Требуется сопоставить физические файлы с данными на диске с записями в таблице по столбцу 'ScanDir ID', чтобы убедиться, что все файлы соответствуют записям в таблице.
Все файлы, которые не имеют соответствующих записей в таблице, должны быть удалены из набора данных. И наоборот, все записи в таблице, которые не имеют соответствующих файлов на диске, должны быть удалены из датасетов.